In [1]:
import torch
import os
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM # 导入你的模型类
from transformers import AutoTokenizer
import lm_eval
import json
import os
# 1. 定义配置和模型保存路径
save_directory = "my_pretrain_model_hf" # 这是你最终要创建的HF标准模型目录
checkpoint_path = "./pretrain_output/latest_checkpoint.pth"

# 2. 像加载模型进行推理一样，先创建一个模型实例
#    你需要知道这个checkpoint对应的配置
config = MiniMindConfig(hidden_size=512, num_hidden_layers=8, use_moe=True) # 假设配置
model = MiniMindForCausalLM(config)



In [8]:
save_directory = "my_pretrain_model_hf"

In [2]:
tokenizer = AutoTokenizer.from_pretrained('model')

In [3]:
class TrainArgs():
    #文件管理
    out_dir = './pretrain_output'
    checkpoint_path = "./pretrain_output/latest_checkpoint.pth"
    data_path = 'data/pretrain_hq.jsonl'
    #神经网络训练管理
    epochs = 2
    batch_size = 32
    accumulation_steps = 4
    learning_rate = 5e-4
    warm_up = 0
    grad_clip = 1
    dtype = 'bfloat16'
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    num_workers = min(8, os.cpu_count()) 
    log_interval = 50
    save_interval = 2000

In [4]:
# 3. 加载你训练好的checkpoint
print(f"Loading checkpoint from: {checkpoint_path}")
checkpoint = torch.load(checkpoint_path, map_location='cpu') # 加载到cpu，更通用


Loading checkpoint from: ./pretrain_output/latest_checkpoint.pth


C:\Users\kkkkkk\AppData\Local\Temp\ipykernel_25652\4223357027.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location='cpu'

In [5]:
# 4. 提取并加载模型权重 (state_dict)
model_state_dict = checkpoint['model']
# 如果权重是半精度的，最好转回全精度以保证兼容性


model.load_state_dict(model_state_dict)

print("Model loaded successfully!")

Model loaded successfully!


In [9]:
os.makedirs(save_directory, exist_ok=True)

# 5. 调用 save_pretrained
#    这个方法会自动为我们完成所有打包工作！
model.save_pretrained(
    save_directory,
    safe_serialization=True # 推荐使用safetensors格式，更安全、加载更快
)

print(f"Model saved in Hugging Face format to: {save_directory}")

Model saved in Hugging Face format to: my_pretrain_model_hf


In [10]:
tokenizer.save_pretrained(save_directory)

('my_pretrain_model_hf\\tokenizer_config.json',
 'my_pretrain_model_hf\\special_tokens_map.json',
 'my_pretrain_model_hf\\chat_template.jinja',
 'my_pretrain_model_hf\\tokenizer.json')